# Testing Deployed LLMs

In [ ]:
!pip install openai==1.6.1 --upgrade --user

### VLLM Completion API

In [1]:
from openai import OpenAI
from rich import print

ip = "20.228.200.49"
model = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = f"http://{ip}:8000/v1"

system_prompt = "What are an expert chef's"
user_prompt = "What is the recipe of mayonnaise explain in detail?"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
# completion = client.completions.create(model=model,
#                                       prompt="What is the recipe of mayonnaise?")
completion = client.completions.create(model=model,
                                      prompt=f"what is your name?")
print("Completion result:", completion)

Completion result:
Completion(
    id='cmpl-360d77a88e0b46af961fa741e70ddeed',
    choices=[
        CompletionChoice(
            finish_reason='length',
            index=0,
            logprobs=None,
            text='\nMeet Mike Slotkin. He is a friend of mine and now'
        )
    ],
    created=1155,
    model='mistralai/Mixtral-8x7B-Instruct-v0.1',
    object='text_completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=16, prompt_tokens=6, total_tokens=22)
)

### VLLM chat API

In [10]:
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = f"http://{ip}:8000/v1"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
    base_url=openai_api_base,
)

# models = client.models.list()
# model = models.data[0].id

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "You name is Avalon"
        }, 
        {
            "role":"assistant",
            "content":"Yes master got it!"
        }, 
        {
            "role": "user",
            "content": "what is your name?"
        },
    ],
    model=model,
)


print("Chat completion results:")
print(chat_completion)

Chat completion results:

ChatCompletion(
    id='cmpl-351202894347480a9ab3233b4972abd4',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=' My name is Avalon, as you have previously stated. How can I assist you today, master?',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1506,
    model='mistralai/Mixtral-8x7B-Instruct-v0.1',
    object='chat.completion',
    system_fingerprint=None,
    usage=CompletionUsage(completion_tokens=23, prompt_tokens=33, total_tokens=56)
)

### Text Generation Inferences Request

In [7]:
import requests

headers = {
    "Content-Type": "application/json",
}

data = {
    'inputs': '[INST] what is the meaning of life[/INST]',
    'parameters': {
        'max_new_tokens': 2000,
    },
}

response = requests.post('http://4.227.161.123:8000/generate', headers=headers, json=data)
print(response.json())

{
    'generated_text': ' The meaning of life is a philosophical question that has been debated for centuries. There 
is no one definitive answer to this question, as it is a deeply personal and subjective matter. Some people believe
that the meaning of life is to seek happiness and fulfillment, while others believe that it is to serve a higher 
power or to contribute to the greater good of society. Ultimately, the meaning of life is a matter of individual 
interpretation and belief.'
}

### Text Generation Inference Streaming

TGI also has streaming support - [Documentation](https://huggingface.co/docs/text-generation-inference/conceptual/streaming)

In [ ]:
from huggingface_hub import AsyncInferenceClient

client = AsyncInferenceClient("http://127.0.0.1:8080")
async for token in await client.text_generation("How do you make cheese?", stream=True):
    print(token)